In [1]:
import os

# clone the repository
if not os.path.exists('MODNet'):
  !git clone https://github.com/ZHKKKe/MODNet
%cd MODNet/

# dowload the pre-trained ckpt for image matting
pretrained_ckpt = 'pretrained/modnet_photographic_portrait_matting.ckpt'
if not os.path.exists(pretrained_ckpt):
  !gdown --id 1mcr7ALciuAsHCpLnrtG_eop5-EYhbCmz \
          -O pretrained/modnet_photographic_portrait_matting.ckpt

C:\Users\cuco2\Desktop\Verano2021\The_Bridge\Archivos de clase\putos peces\MODNet


In [2]:
input_folder = "../Imagenes normalizadas"
output_folder = "../sin fondo"

In [9]:
import sys
print(sys.executable)

C:\Users\cuco2\desktop\Verano2021\The_Bridge\env\Scripts\python.exe


In [13]:
!pip install torch
!pip install torchvision

You should consider upgrading via the 'c:\users\cuco2\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


  Using cached torchvision-0.11.1-cp39-cp39-win_amd64.whl (984 kB)

You should consider upgrading via the 'c:\users\cuco2\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.



  Attempting uninstall: pillow
    Found existing installation: Pillow 8.3.0
    Uninstalling Pillow-8.3.0:
      Successfully uninstalled Pillow-8.3.0


In [20]:
!pip uninstall torch

Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html


ERROR: Could not find a version that satisfies the requirement torchaudio===0.10.0+cu113 (from versions: 0.7.1, 0.7.2, 0.8.0, 0.8.1, 0.9.0, 0.9.1, 0.10.0)
ERROR: No matching distribution found for torchaudio===0.10.0+cu113
You should consider upgrading via the 'c:\users\cuco2\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


^C


In [21]:
!pip3 install torch==1.10.0+cu113 torchvision==0.11.1+cu113 torchaudio===0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
  Using cached https://download.pytorch.org/whl/cu113/torch-1.10.0%2Bcu113-cp39-cp39-win_amd64.whl (2442.3 MB)
  Using cached https://download.pytorch.org/whl/cu113/torchvision-0.11.1%2Bcu113-cp39-cp39-win_amd64.whl (3.2 MB)


ERROR: Could not find a version that satisfies the requirement torchaudio===0.10.0+cu113 (from versions: 0.7.1, 0.7.2, 0.8.0, 0.8.1, 0.9.0, 0.9.1, 0.10.0)
ERROR: No matching distribution found for torchaudio===0.10.0+cu113
You should consider upgrading via the 'c:\users\cuco2\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [16]:
!python -m demo.image_matting.colab.inference \
        --input-path ../Imagenes_normalizadas \
        --output-path ../sin_fondo \
        --ckpt-path ./pretrained/modnet_photographic_portrait_matting.ckpt

Traceback (most recent call last):
  File "C:\Users\cuco2\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\cuco2\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\cuco2\Desktop\Verano2021\The_Bridge\Archivos de clase\putos peces\MODNet\demo\image_matting\colab\inference.py", line 47, in <module>
    modnet = nn.DataParallel(modnet).cuda()
  File "C:\Users\cuco2\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\module.py", line 680, in cuda
    return self._apply(lambda t: t.cuda(device))
  File "C:\Users\cuco2\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\module.py", line 570, in _apply
    module._apply(fn)
  File "C:\Users\cuco2\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\module.py", line 570, in _apply
    module._apply(fn)
  File "

In [18]:
import numpy as np
from PIL import Image

def combined_display(image, matte):
  # calculate display resolution
  w, h = image.width, image.height
  rw, rh = 800, int(h * 800 / (3 * w))
  
  # obtain predicted foreground
  image = np.asarray(image)
  if len(image.shape) == 2:
    image = image[:, :, None]
  if image.shape[2] == 1:
    image = np.repeat(image, 3, axis=2)
  elif image.shape[2] == 4:
    image = image[:, :, 0:3]
  matte = np.repeat(np.asarray(matte)[:, :, None], 3, axis=2) / 255
  foreground = image * matte + np.full(image.shape, 255) * (1 - matte)
  
  # combine image, foreground, and alpha into one line
  combined = np.concatenate((image, foreground, matte * 255), axis=1)
  combined = Image.fromarray(np.uint8(combined)).resize((rw, rh))
  return combined

# visualize all images
image_names = os.listdir(input_folder)
for image_name in image_names:
  matte_name = image_name.split('.')[0] + '.png'
  image = Image.open(os.path.join(input_folder, image_name))
  matte = Image.open(os.path.join(output_folder, matte_name))
  display(combined_display(image, matte))
  print(image_name, '\n')


FileNotFoundError: [Errno 2] No such file or directory: '../sin fondo\\Angelfish.png'